## Required Libraries/Packages

In [155]:
import xml.etree.ElementTree as ET
#import xml.etree.cElementTree as ET
from bs4 import BeautifulSoup
import pandas as pd
import re

## XML Conversion to Pandas DataFrame Object

In [154]:
def xml_retrieve_to_dataframe(search_tag):
    with open(r'S:\Code\School\WGU_DataAnalyst_NanoDegree\02 - Data Wrangling with MongoDB\nevada-latest.osm',encoding="utf8") as osm_file:
        nodes = []
        node_tags = []
        for event, elem in ET.iterparse(osm_file):
            for items in elem:
                    if items.tag == search_tag:
                        nodes.append(items.attrib)
                        if list(items):
                            for x in list(items):
                                temp_dict = {}
                                temp_dict.update(x.attrib)
                                temp_dict['type'] = x.tag
                                temp_dict['id'] = items.attrib['id']
                                node_tags.append(temp_dict)
        df = pd.DataFrame(nodes)
        tag_df = pd.DataFrame(node_tags)
        return df, tag_df

## Node Related Datasets

In [156]:
nodes, nodes_tags = xml_retrieve_to_dataframe('node')

In [157]:
nodes.head()

,changeset,id,lat,lon,timestamp,version
0,0,26798725,39.4433397,-120.0117505,2012-08-03T15:23:02Z,3
1,0,26798726,39.4407936,-120.0175432,2012-08-03T15:23:02Z,3
2,0,26798727,39.4317906,-120.0319132,2009-01-24T17:21:24Z,2
3,0,26798728,39.4238716,-120.035895,2012-09-03T22:29:35Z,4
4,0,26798729,39.4166984,-120.0300012,2009-01-24T17:09:06Z,2


In [98]:
nodes_tags.head()

,id,k,type,v
0,137043163,highway,tag,motorway_junction
1,137043163,ref,tag,40
2,137073542,crossing,tag,traffic_signals
3,137073542,highway,tag,traffic_signals
4,137093920,highway,tag,motorway_junction


## Way Related Datasets

In [115]:
ways, ways_tags = xml_retrieve_to_dataframe('way')

In [116]:
ways.head()

,changeset,id,timestamp,uid,user,version,visible
0,56309289,14278848,2018-02-13T00:51:26Z,4018842,Stephen214,11,true
1,56309289,14279688,2018-02-13T00:51:26Z,4018842,Stephen214,15,true
2,57579072,14280720,2018-03-27T19:42:48Z,5288591,Irina Karachevtseva,2,true
3,37055103,14281281,2016-02-07T08:20:30Z,110838,Bernhard W,9,true
4,64992575,14282209,2018-11-29T01:04:59Z,8107451,njtbusfan,17,true


In [117]:
ways_tags.head()

,id,k,ref,type,v
0,14278848,NaN,967221850,nd,NaN
1,14278848,NaN,473360516,nd,NaN
2,14278848,NaN,137037541,nd,NaN
3,14278848,NaN,137037544,nd,NaN
4,14278848,NaN,137037546,nd,NaN


In [118]:
ways_nodes = ways_tags.loc[ways_tags['type'] == 'nd', :]
ways_tags = ways_tags.loc[ways_tags['type'] != 'nd', :]

In [119]:
ways_nodes.drop(columns=['k', 'type', 'v'], inplace=True)
ways_nodes.head()

,id,ref
0,14278848,967221850
1,14278848,473360516
2,14278848,137037541
3,14278848,137037544
4,14278848,137037546


In [120]:
ways_tags.drop(columns=['ref'], inplace=True)
ways_tags.head()

,id,k,type,v
24,14278848,bicycle,tag,no
25,14278848,destination:street,tag,Flamingo Road East
26,14278848,highway,tag,motorway_link
27,14278848,oneway,tag,yes
28,14278848,tiger:cfcc,tag,A63


## Data Cleaning/Munging

Discover what values may be good candidates for cleaning (i.e. lots of values) and have values that can be recoded.

In [121]:
for col_name in ways_tags['k'].unique():
    print(col_name, ' - ', ways_tags.loc[ways_tags['k'] == col_name, :].count()['v'])

bicycle  -  99
destination:street  -  7
highway  -  2394
oneway  -  639
tiger:cfcc  -  284
tiger:county  -  282
tiger:reviewed  -  128
lanes  -  109
service  -  481
source  -  220
name  -  884
ref  -  59
surface  -  162
tiger:name_base  -  248
tiger:name_direction_prefix  -  43
tiger:name_type  -  220
maxspeed  -  108
bridge  -  97
FIXME  -  19
layer  -  165
access  -  71
old_name  -  65
tiger:separated  -  6
tiger:source  -  10
tiger:tlid  -  10
name_1  -  20
tiger:name_base_1  -  21
tiger:name_type_1  -  14
tiger:name_direction_prefix_1  -  8
postal_code  -  2
name:ko  -  1
name:ru  -  1
natural  -  84
tourism  -  58
website  -  84
wheelchair  -  10
tiger:upload_uuid  -  7
amenity  -  213
building  -  794
building:levels  -  172
fee  -  15
parking  -  61
addr:housenumber  -  40
addr:postcode  -  48
addr:street  -  40
height  -  51
name:en  -  5
name:zh  -  6
phone  -  47
leisure  -  182
sport  -  51
landuse  -  119
tunnel  -  32
building:colour  -  8
building:material  -  5
building:

Yes/No now replace with True/False values. These could be boolean if data was of uniform type.

In [151]:
ways_tags['v'].replace(yes_no_dict, inplace=True)

In [153]:
#Ensure that data is properly cleaned
ways_tags.head(10)

,id,k,type,v
24,14278848,bicycle,tag,False
25,14278848,destination:street,tag,Flamingo Road East
26,14278848,highway,tag,motorway_link
27,14278848,oneway,tag,True
28,14278848,tiger:cfcc,tag,A63
29,14278848,tiger:county,tag,"Clark, NV"
33,14279688,bicycle,tag,False
34,14279688,destination:street,tag,Spring Mountain Road
35,14279688,highway,tag,motorway_link
36,14279688,oneway,tag,True


In [4]:
import sqlite3 as sq
connection = sq.connect(r'S:\Code\School\WGU_DataAnalyst_NanoDegree\02 - Data Wrangling with MongoDB\MapData.db')

cursor = connection.cursor()

query = '''select * from nodes'''
cursor.execute(query)

result = cursor.fetchall()

In [5]:
result

[]